In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import requests
import numpy as np
from sklearn.preprocessing import MinMaxScaler

sess = tf.Session()

In [2]:
#get data
birthdata_url = 'http://springer.bme.gatech.edu/Ch17.Logistic/Logisticdat/birthweight.dat'
birth_file = requests.get(birthdata_url)
birth_data = birth_file.text.split('\r\n')
print(len(birth_data))

birth_header = [x for x in birth_data[0].split('\t')]
print(birth_header)

birth_data = [[ x for x in y.split('\t')] for y in birth_data[1:-2]]
print(len(birth_data))

y_vals = np.array([x[8] for x in birth_data])
print(len(y_vals))
print(y_vals[0])

#enumerate example
# >>>seasons = ['Spring', 'Summer', 'Fall', 'Winter']
# >>> list(enumerate(seasons))
# [(0, 'Spring'), (1, 'Summer'), (2, 'Fall'), (3, 'Winter')]
cols = ['AGE', 'LWT', 'RACE', 'SMOKE', 'PTL', 'HT', 'UI']
x_vals = np.array([[x[ix] for ix, feature in enumerate(birth_header) if feature in cols ] for x in birth_data])
print(len(x_vals))
print(x_vals[0])

# 列表推导式
# 列表生成式即List Comprehensions，是Python内置的非常简单却强大的可以用来创建list的生成式。
# 它的结构是在一个中括号里包含一个表达式，然后是一个for语句，然后是 0 个或多个 for 或者 if 语句。
# 那个表达式可以是任意的，意思是你可以在列表中放入任意类型的对象。
# 返回结果将是一个新的列表，在这个以 if 和 for 语句为上下文的表达式运行完成之后产生。
#  列表推导式的执行顺序：各语句之间是嵌套关系，左边第二个语句是最外层，依次往右进一层，左边第一条语句是最后一层。

# [x*y for x in range(1,5) if x > 2 for y in range(1,4) if y < 3]
# 他的执行顺序是:

# for x in range(1,5)
#     if x > 2
#         for y in range(1,4)
#             if y < 3
#                 x*y

192
['LOW', 'AGE', 'LWT', 'RACE', 'SMOKE', 'PTL', 'HT', 'UI', 'BWT']
189
189
709
189
['28' '113' '1' '1' '1' '0' '1']


In [3]:
#convert typeA to typeB
#numpy.ndarray.astype
#Copy of the array, cast to a specified type.

# >>> x = np.array([1, 2, 2.5])
# >>> x.astype(int)
# array([1, 2, 2])

x_vals.astype(np.float32)
y_vals.astype(np.float32)

array([  709.,  1021.,  1135.,  1330.,  1474.,  1588.,  1588.,  1701.,
        1729.,  1790.,  1818.,  1885.,  1893.,  1899.,  1928.,  1928.,
        1928.,  1936.,  1970.,  2055.,  2055.,  2082.,  2084.,  2084.,
        2100.,  2125.,  2126.,  2187.,  2187.,  2211.,  2225.,  2240.,
        2240.,  2282.,  2296.,  2296.,  2301.,  2325.,  2353.,  2353.,
        2367.,  2381.,  2381.,  2381.,  2395.,  2410.,  2410.,  2414.,
        2424.,  2438.,  2442.,  2450.,  2466.,  2466.,  2466.,  2495.,
        2495.,  2495.,  2495.,  2523.,  2551.,  2557.,  2594.,  2600.,
        2622.,  2637.,  2637.,  2663.,  2665.,  2722.,  2733.,  2750.,
        2750.,  2769.,  2769.,  2778.,  2782.,  2807.,  2821.,  2835.,
        2835.,  2836.,  2863.,  2877.,  2877.,  2906.,  2920.,  2920.,
        2920.,  2920.,  2948.,  2948.,  2977.,  2977.,  2977.,  2977.,
        2992.,  3005.,  3033.,  3042.,  3062.,  3062.,  3062.,  3076.,
        3076.,  3080.,  3090.,  3090.,  3090.,  3100.,  3104.,  3132.,
      

In [4]:
seed =3 
tf.set_random_seed(seed)
np.random.seed(seed)
batch_size = 100

In [5]:
# Generate a uniform random sample from np.arange(5) of size 3:
# >>> np.random.choice(5, 3)
# array([0, 3, 4])
# >>> #This is equivalent to np.random.randint(0,5,3)

# Generate a uniform random sample from np.arange(5) of size 3 without replacement:
# >>> np.random.choice(5, 3, replace=False)
# array([3,1,0])
# >>> #This is equivalent to np.random.permutation(np.arange(5))[:3]

train_indices = np.random.choice(len(x_vals), round(len(x_vals)*0.8), replace = False)
test_indices = np.array(list(set(range(len(x_vals))) -  set(train_indices)))

x_vals_train = x_vals[train_indices]
x_vals_test = x_vals[test_indices]
y_vals_train = y_vals[train_indices]
y_vals_test = y_vals[test_indices]

In [6]:
#关于归一化，标准化由于英文翻译造成的混乱，总结如下：

# Rescaling →　「归一化：使数据分布在0～1之间，从椭圆变为圆形，便于梯度下降」（下面图片中：第一种）
#Standardization →　「标准化：使数据处理后符合标准正态分布」（下面图片中：第三种）
#Regularization →　 「正则化」（如L1，L2，不用在这里所说的特征处理上）

from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "normfor4.png", width=400, height=200)

In [7]:
#sklearn  https://scikit-learn.org/stable/modules/preprocessing.html#preprocessing-normalization
# 标准化操作  StandardScaler() （6.3.1. Standardization, or mean removal and variance scaling）  
#归一化操作   MinMaxScaler() （6.3.1.1. Scaling features to a range） →　归一化是特殊的标准化，相当于 （均值=0，方差=1） 的情况

min_max_scaler = MinMaxScaler()

x_vals_train = min_max_scaler.fit_transform(x_vals_train)
x_vals_test  = min_max_scaler.fit_transform(x_vals_test)

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype <U3 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [8]:
# def normalize_cols(m):
#     col_max = m.max(axis=0)
#     col_min = m.min(axis=0)
#     return (m - col_min)/ (col_max - col_min)

# x_vals_train = np.nan_to_num(normalize_cols(x_vals_train))
# x_vals_test = np.nan_to_num(normalize_cols(x_vals_test))

In [9]:
#tf.random_normal(shape,mean=0.0,stddev=1.0,dtype=tf.float32,seed=None,name=None)
# shape：一个一维整数张量或Python数组。代表张量的形状。
# mean：数据类型为dtype的张量值或Python值。是正态分布的均值。
# stddev：数据类型为dtype的张量值或Python值。是正态分布的标准差。
# dtype： 输出的数据类型。
# seed：一个Python整数。是随机种子。
# name： 操作的名称(可选)

#initialize w and b
def init_weight(shape, st_dev):
    weight = tf.Variable(tf.random_normal(shape, stddev= st_dev))
    return weight

def init_bias(shape, st_dev):
    bias = tf.Variable(tf.random_normal(shape, stddev=st_dev))
    return bias

In [10]:
#initialize placeholder
x_data = tf.placeholder(shape=[None,7], dtype=tf.float32)
y_target = tf.placeholder(shape=[None,1],dtype=tf.float32)

In [11]:
#full connected
def fully_connected(input_layer, weights, biases):
    layer = tf.add(tf.matmul(input_layer, weights), biases)
    return tf.nn.relu(layer)

In [12]:
# 三个隐藏层的神经元个数分别为25，10，3
# 第一隐藏层之间的变量：25*7（w1）+25(b1)=200
# 第二隐藏层之间的变量：25*10（w2）+10（b2）=260
# 第三隐藏层之间的变量：10*3（w3）+3（b3）=33
# 输出层的变量：3*1（w4）+1(b4)=4
# 需要拟合的变量总共为：200+260+33+4=497

#Create first layer(25 hidden nodes)
weight1 = init_weight(shape=[7,25], st_dev=10.0)
bias1 = init_bias(shape=[25], st_dev=10.0)
layer1 = fully_connected(x_data, weight1, bias1)

#Create second layer(10 hidden nodes)
weight2 = init_weight(shape=[25,10], st_dev=10.0)
bias2 = init_bias(shape=[10], st_dev=10.0)
layer2 = fully_connected(layer1, weight2, bias2)

#Create third layer(3 hidden nodes)
weight3 = init_weight(shape=[10,3], st_dev=10.0)
bias3 = init_bias(shape=[3], st_dev=10.0)
layer3 = fully_connected(layer2, weight3, bias3)

#Create output layer(1 output values)
weight4 = init_weight(shape=[3,1], st_dev=10.0)
bias4 = init_bias(shape=[1], st_dev=10.0)
final_output = fully_connected(layer3, weight4, bias4)

In [13]:
#Loss(L1) , Optimizer(AdamOptimizer)
loss = tf.reduce_mean(tf.abs(y_target-final_output))
my_opt = tf.train.AdamOptimizer(0.05)
train_step= my_opt.minimize(loss)
init =  tf.global_variables_initializer()
sess.run(init)

In [16]:
#Initialize the loss vectors
loss_vec = []
test_loss = []
batch_size = 25

for i in range(500):
    # Choose random indices for batch selection
    rand_index = np.random.choice(len(x_vals_train), size=batch_size)
    # Get random batch
    rand_x = x_vals_train[rand_index]
    rand_y = np.transpose([y_vals_train[rand_index]])
    # Run the training step
    sess.run(train_step,feed_dict={x_data:rand_x, y_target:rand_y})
    # Get and store the train loss
    train_temp_loss = sess.run(loss,feed_dict={x_data:rand_x, y_target:rand_y})
    loss_vec.append(train_temp_loss)
    # Get and store the test loss
    test_temp_loss = sess.run(loss,feed_dict={x_data:x_vals_test, y_target:np.transpose([y_vals_test])})
    test_loss.append(test_temp_loss)
    
    if(i+1)%25==0:
        print('Generation: '+str(i+1)+' Loss ='+str(train_temp_loss))

Generation: 25 Loss =398.422
Generation: 50 Loss =455.814
Generation: 75 Loss =624.154
Generation: 100 Loss =505.198
Generation: 125 Loss =501.907
Generation: 150 Loss =449.634
Generation: 175 Loss =366.31
Generation: 200 Loss =454.773
Generation: 225 Loss =550.58
Generation: 250 Loss =511.725
Generation: 275 Loss =425.742
Generation: 300 Loss =341.477
Generation: 325 Loss =347.954
Generation: 350 Loss =314.968
Generation: 375 Loss =460.222
Generation: 400 Loss =480.144
Generation: 425 Loss =442.029
Generation: 450 Loss =448.594
Generation: 475 Loss =573.863
Generation: 500 Loss =292.359


In [ ]:
# actuals = np.array([x[1] for x in birth_data]) 
# test_actuals = actuals[test_indices] 
# train_actuals = actuals[train_indices] 
# test_preds = [x[0] for x in sess.run(final_output, feed_dict={x_data: x_vals_test})] 
# train_preds = [x[0] for x in sess.run(final_output, feed_dict={x_data: x_vals_train})] 
# test_preds = np.array([1.0 if x<2500.0 else 0.0 for x in test_preds]) 
# train_preds = np.array([1.0 if x<2500.0 else 0.0 for x in train_preds]) 
# # Print out accuracies 
# test_acc = np.mean([x==y for x,y in zip(test_preds, test_actuals)]) 
# train_acc = np.mean([x==y for x,y in zip(train_preds, train_actuals)]) 
# print('On predicting the category of low birthweight from regression output (<2500g):') 
# print('Test Accuracy: {}'.format(test_acc)) 
# print('Train Accuracy: {}'.format(train_acc)) 